## Baseline Calculation (Dont run this notebook, execute any of the experiments instead)

In [1]:
import networkx as nx
import numpy as np
from collections import defaultdict
import itertools

In [2]:
def find_and_add_entry_node(graph):
    # First identify the original root nodes
    original_roots = [n for n, deg in graph.in_degree() if deg == 0]
    
    if len(original_roots) > 1:
        # add virtual entry node
        entry = 0 
        graph.add_node(entry)
        for r in original_roots:
            graph.add_edge(entry, r, weight=DEFAULT_WEIGHT_VALUE)
        return entry, graph, original_roots
    else:
        entry = original_roots[0]
        return entry, graph, original_roots

In [3]:
def merge_targets_with_multi_edges(orig_graph):
    targets = []
    
    for node, out_degree in orig_graph.out_degree():
        if out_degree == 0:
            targets.append(node)
    
    if len(targets) <= 1:
        return orig_graph

    merged_label = "c(" + ",".join(str(t) for t in targets) + ")"

    newG = nx.MultiDiGraph()

    non_targets = []
    for node in orig_graph.nodes():
        if node not in targets:
            non_targets.append(node)
            
    for node in non_targets:
        newG.add_node(node)
        
    newG.add_node(merged_label)
    
    pred_target_edges = {}

    for u, v, data in orig_graph.edges(data=True):
        if v in targets:
            if u not in pred_target_edges:
                pred_target_edges[u] = []
            weight = data.get('weight', DEFAULT_WEIGHT_VALUE)
            pred_target_edges[u].append((weight, v))

    for u, edges in pred_target_edges.items():
        weight_counts = {}
        for weight, _ in edges:
            weight_counts[weight] = weight_counts.get(weight, 0) + 1

        for weight, count in weight_counts.items():
            for _ in range(count):
                newG.add_edge(u, merged_label, weight=weight)
    
    for u, v, data in orig_graph.edges(data=True):
        if v not in targets and u not in targets:
            newG.add_edge(u, v, **data)

    return newG

In [ ]:
entry, attack_graph, original_roots = find_and_add_entry_node(attack_graph)
attack_graph = merge_targets_with_multi_edges(attack_graph)

# Find targets manually since your function doesn't return them
target_list = [n for n in attack_graph.nodes() if attack_graph.out_degree(n) == 0]

In [ ]:
# Enumerate all paths (attack strategies) and defense points
def setup_game(attack_graph, entry, target_list):
    target = target_list[0]  # Single target after condensation
    
    # Get all simple paths from entry to target
    routes = list(nx.all_simple_paths(attack_graph, entry, target))
    
    # Get all nodes from all routes
    V = list(set(node for route in routes for node in route))
    
    # Node ordering from topological sort
    try:
        node_order = list(nx.topological_sort(attack_graph))
        node_order = [n for n in node_order if n in V]
    except:
        node_order = V
    
    # Defender action space (exclude trivial nodes)
    if 'as1' not in globals():
        as1 = [n for n in V if n not in [entry, "attacker_entry_node"] + target_list]
    
    as2 = routes  # Attacker action space
    
    # Adversary list (potential starting points)
    adv_list = [n for n in V if n not in [entry] + target_list]
    
    return V, routes, as1, as2, adv_list, target, node_order

V, routes, as1, as2, adv_list, target, node_order = setup_game(attack_graph, entry, target_list)

In [ ]:
# Set default rates if not externally defined
if 'defense_rate_list' not in globals():
    defense_rate_list = [0]
if 'attack_rate_list' not in globals():
    attack_rate_list = [0]

starting_points = [n for n in V if n not in target_list]

In [ ]:
for defense_rate in defense_rate_list:
    for attack_rate in attack_rate_list:
        
        U = {node: 0.0 for node in V}
        
        for i in as1:
            for j in starting_points:
                
                try:
                    route = nx.shortest_path(attack_graph, j, target)
                except:
                    continue
                
                pdf_d = random_steps(route, attack_rate, defense_rate, attack_graph)
                
                try:
                    cut_point = route.index(i) + 1
                except ValueError:
                    cut_point = len(route)
                
                if sum(pdf_d[:cut_point]) == 0:
                    payoff_distr = [0] * cut_point
                    payoff_distr[cut_point-1] = 1
                else:
                    total = sum(pdf_d[:cut_point])
                    payoff_distr = [p/total for p in pdf_d[:cut_point]]
                
                for idx, node in enumerate(route[:cut_point]):
                    prob = payoff_distr[idx] * (1/len(as1)) * (1/len(V))
                    U[node] += prob
        
        print(f"Defense rate = {defense_rate}, Attack rate = {attack_rate}")
        print(f"Target {target} hit probability: {U[target]:.6f}\n")